# SUPG Advection Example: Dispersion and Phase Speed Diagrams

In [ ]:
# dependencies
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.develop(path="../..")
using Plots
using LFAToolkit
using LinearAlgebra 

In [ ]:
# setup
p = 2
q = p
dimension = 1
mapping = nothing
collocate = false
basis = TensorH1LagrangeBasis(p, q, 1, 1, collocatedquadrature = collocate, mapping = mapping)

mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

# advective speed
c = 1.0
# Tau scaling for SUPG, 0 returns Galerkin method
τ = 0.5 / (p - 1)

In [ ]:
# SUPG advection weakform 
function supgadvectionweakform(U::Matrix{Float64}, w::Array{Float64})
    u = U[1, :]
    du = U[2, :]
    dv = (c * u - c * τ * (c * du)) * w[1]
    return [dv]
end

# SUPG advection operator
inputs = [
    OperatorField(
        basis,
        [EvaluationMode.interpolation, EvaluationMode.gradient],
        "advected field",
    ),
    OperatorField(basis, [EvaluationMode.quadratureweights], "quadrature weights"),
]
outputs = [OperatorField(basis, [EvaluationMode.gradient])]
supgadvection = Operator(supgadvectionweakform, mesh, inputs, outputs)

# SUPG mass weakform and mass operator
function supgmassweakform(udot::Array{Float64}, w::Array{Float64})
    v = udot * w[1]
    dv = c * τ * udot * w[1]
    return ([v; dv],)
end
supgmass = Operator(
    supgmassweakform,
    mesh,
    [
        OperatorField(basis, [EvaluationMode.interpolation], "u_t"),
        OperatorField(basis, [EvaluationMode.quadratureweights], "quadrature weights"),
    ],
    [OperatorField(basis, [EvaluationMode.interpolation, EvaluationMode.gradient])],
)

In [ ]:
# compute full operator symbols
numbersteps = 100
maxeigenvalue = 0
θ_min = 0
θ_max = (p-1) * π
θ_range = LinRange(θ_min, θ_max, numbersteps)

# compute and plot dispersion and phase speed
# -- 1D --
function advection_supg_symbol(θ_range)
    A = computesymbols(supgadvection, [θ_range]) * 2 # transform from reference to physical on dx=1 grid
    M = computesymbols(supgmass, [θ_range]) # mass matrix
    return sort(imag.(eigvals(-M \ A)))
end

eigenvalues = hcat(advection_supg_symbol.(θ_range)...)'

plot1 = plot(θ_range / π, eigenvalues ./ π, linewidth = 3) # Dispersion diagram
        plot!(
        identity,
        xlabel = "θ/π",
        ylabel = "Eigenvalues",
        label = "exact",
        legend = :none,
        color = :black,
        title = "Dispersion relation, p=$p, collocate=$collocate, τ=$τ",
)
plot2 = plot(θ_range / π, eigenvalues ./ θ_range, linewidth = 3) # Phase speed diagram
        plot!(
        one,
        xlabel = "θ/π",
        ylabel = "Eigvalues/θ",
        legend = :none,
        color = :black,
        title = "Phase speed, p=$p, collocate=$collocate, τ=$τ",
)
plot!(plot1, plot2, layout = (1, 2), size = (1050, 450))